# **1. Load Data**

In [21]:
import re
import html
import contractions
import requests



class DataLoader:
    def __init__(self, url_en, url_vi):
        self.__en_data = self.__load_data(url_en,'en')
        self.__vi_data = self.__load_data(url_vi,'vi')

    def __text_preprocessing(self, text:str, language:str = 'en'):
        text = html.unescape(text)
        text = re.sub(' +', ' ', text)

        if language == 'en':
            from underthesea import sent_tokenize
            text = contractions.fix(text)
        else:
            from nltk import sent_tokenize

        return [sentence for sentence in sent_tokenize(text)]

    def __load_data(self, url, language:str):
        return [self.__text_preprocessing(line,language) for line in requests.get(url).text.splitlines()]

    @property
    def vi(self):
        return self.__vi_data

    @property
    def en(self):
        return self.__en_data

In [2]:
url = "https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/"

train = DataLoader(url +'train.en',url +'train.vi')
test = DataLoader(url + 'tst2013.en',url + 'tst2013.vi')

In [3]:
for i in range(10,20):
    print(i,train.vi[i])
print(len(train.en))

10 ['Mỗi năm , hơn 15,000 nhà khoa học đến San Francisco để tham dự hội nghị này .']
11 ['Mỗi một khoa học gia đều thuộc một nhóm nghiên cứu , và mỗi nhóm đều nghiên cứu rất nhiều đề tài đa dạng .']
12 ['Với chúng tôi , tại Cambridge , các đề tài thay đổi từ sự dao động của El Niño , vốn có tác động đến thời tiết và khí hậu , sự đồng hoá thông tin từ vệ tinh , khí thải từ những cánh đồng nhiên liệu sinh học , tình cờ lại là đề tài tôi nghiên cứu .']
13 ['Mỗi lĩnh vực nghiên cứu lại chia ra những lĩnh vực nhỏ hơn , và những nghiên cứu sinh có bằng tiến sĩ , như tôi , phải nghiên cứu những đề tài vô cùng cụ thể , cụ thể như chỉ vài quy trình hay vài phân tử .']
14 ['Một trong số những phân tử tôi nghiên cứu tên là isoprene .', 'Đây .', 'Nó là một phân tử hữu cơ nhỏ .', 'Có thể các bạn cũng chưa từng nghe tên .']
15 ['Trọng lượng của một chiếc kẹp giấy vào khoảng 900 zeta-illion -- 10 mũ 21 -- phân tử isoprene .']
16 ['Dù trọng lượng phân tử rất nhỏ , thế nhưng lượng isoprene được thải và

# **2. Vocab**

In [4]:
from typing import Iterator, List, Optional
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from underthesea import word_tokenize

class Language:
    def __init__(self, name: str = 'en'):
        if name == 'en':
            self.__name = 'en'
            self.__tokenizer = get_tokenizer('spacy', language ='en_core_web_sm') 
        else:
            self.__name = 'vi'
            self.__tokenizer = word_tokenize

        self.__vocab = None

    def __yield_tokens(self, data):
        for line in data:
            for sentence in line:
                result = self.__tokenizer(sentence)
                if result is not None:
                    yield result  

    def make_vocab(self, train_iter: Iterator, min_freq:int = 1,specials: Optional[List[str]] = None, default_idx:int = 0):
        self.__vocab = build_vocab_from_iterator(self.__yield_tokens(train_iter), min_freq, specials)
        self.__vocab.set_default_index(default_idx)

    @property
    def name(self):
        return self.__name

    @property
    def vocab(self):
        return self.__vocab
    
    @property
    def tokenizer(self):
        return self.__yield_tokens

In [5]:
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
specials = ["<unk>", "<pad>", "<sos>", "<eos>"]

Vi = Language('vi')
En = Language('en')
Vi.make_vocab(train.vi,3,specials,UNK_IDX)
En.make_vocab(train.en,3,specials,UNK_IDX)

In [6]:
print("vocab size vi:", len(Vi.vocab.get_itos()))
print("vocab size en:", len(En.vocab.get_itos()))
for word in Vi.vocab.get_itos()[:5]:
    print(word,Vi.vocab[word])

vocab size vi: 18141
vocab size en: 23974
<unk> 0
<pad> 1
<sos> 2
<eos> 3
, 4


In [27]:
contractions.fix("it 's")


'it is'

In [23]:
for text in En.tokenizer(train.en[10:15]):
    print(text)

['Over', '15,000', 'scientists', 'go', 'to', 'San', 'Francisco', 'every', 'year', 'for', 'that', '.']
['And', 'every', 'one', 'of', 'those', 'scientists', 'is', 'in', 'a', 'research', 'group', ',', 'and', 'every', 'research', 'group', 'studies', 'a', 'wide', 'variety', 'of', 'topics', '.']
['For', 'us', 'at', 'Cambridge', ',', 'it', "'s", 'as', 'varied', 'as', 'the', 'El', 'Niño', 'oscillation', ',', 'which', 'affects', 'weather', 'and', 'climate', ',', 'to', 'the', 'assimilation', 'of', 'satellite', 'data', ',', 'to', 'emissions', 'from', 'crops', 'that', 'produce', 'biofuels', ',', 'which', 'is', 'what', 'I', 'happen', 'to', 'study', '.']
['And', 'in', 'each', 'one', 'of', 'these', 'research', 'areas', ',', 'of', 'which', 'there', 'are', 'even', 'more', ',', 'there', 'are', 'PhD', 'students', ',', 'like', 'me', ',', 'and', 'we', 'study', 'incredibly', 'narrow', 'topics', ',', 'things', 'as', 'narrow', 'as', 'a', 'few', 'processes', 'or', 'a', 'few', 'molecules', '.']
['And', 'one', '

# **3. Data Preprocessing**

In [ ]:
def data_preprocessing(data:List[List], lang: Language):
    pass

# **4. Model**